
# Design Patterns en Python (Orienté Quant/Trading) — **Notebook Complet**

**Objectifs**
- Comprendre quand et pourquoi utiliser des patterns (sans sur‑designer).
- Maîtriser **Strategy**, **Factory**, **Observer**, **Adapter**, **Facade**, **Singleton**, **Dependency Injection**, **Template Method**.
- Connaître les **pièges** et **bonnes pratiques** en Python (typing, tests, logging).
- Exercices pratiques + mini‑projet **OMS jouet**.

> Toutes les formules sont en `$$ ... $$` si besoin. Le focus ici est l'architecture Python claire et testable.



## 0) Pourquoi les patterns ?

- Structurer pour **itérer vite** et **tester facilement**.
- **Découpler** les responsabilités (remplacer une stratégie/broker sans casser le reste).
- Éviter le **code spaghetti** et le **copier‑coller**.
- En Python, rester **pragmatique** : composition et fonctions first‑class > over‑engineering.



## 1) Strategy (+ Factory)

**Idée** : encapsuler des algorithmes interchangeables derrière une interface commune.

**Quand** : tu veux **switcher** rapidement entre Mean‑Reversion / Momentum / etc., **sans modifier** le rester du code (backtest, OMS…).


In [ ]:

from abc import ABC, abstractmethod

class Strategy(ABC):
    @abstractmethod
    def signal(self, series): ...

class MeanReversion(Strategy):
    def signal(self, series):
        m = sum(series) / len(series)
        return "BUY" if series[-1] < m else "SELL"

class Momentum(Strategy):
    def signal(self, series):
        return "BUY" if series[-1] > series[-2] else "SELL"

def strategy_factory(kind: str) -> Strategy:
    table = {"mr": MeanReversion, "momo": Momentum}
    try:
        return table[kind]()
    except KeyError:
        raise ValueError(f"Unknown strategy {kind!r}")

s = strategy_factory("mr")
print("signal:", s.signal([100, 101, 99, 100]))



**Notes & pièges** :
- Commence simple (fonctions), introduis **Strategy** quand ça paie.
- Utilise `abc.ABC` + `@abstractmethod` pour **documenter** l'interface.
- Tests unitaires par implémentation (contrats communs).



## 2) Observer (pub/sub)

**Idée** : un **sujet** notifie des **observateurs** (listeners).  
**Quand** : un **feed** (ticks/événements) doit informer **plusieurs consommateurs** (stratégies, logs, risk).


In [ ]:

class Subject:
    def __init__(self): self._subs = []
    def subscribe(self, obs): self._subs.append(obs)
    def unsubscribe(self, obs): 
        if obs in self._subs: self._subs.remove(obs)
    def notify(self, data):
        for o in list(self._subs):  # copie défensive
            try:
                o.update(data)
            except Exception as e:
                print("observer error:", e)

class StrategyObserver:
    def __init__(self, name): self.name = name
    def update(self, tick):
        print(f"[{self.name}] tick={tick}")

feed = Subject()
s1, s2 = StrategyObserver("S1"), StrategyObserver("S2")
feed.subscribe(s1); feed.subscribe(s2)
for px in [100.0, 100.2, 99.9]:
    feed.notify(px)
feed.unsubscribe(s1)
feed.notify(101.0)



**Variantes** :
- File asynchrone (`queue.Queue`) pour haut débit (ne pas bloquer le thread producteur).
- Version `asyncio` si les consommateurs font des I/O réseau.



## 3) Adapter

**Idée** : **uniformiser** des APIs de brokers/vendors **différentes** derrière **UNE** interface interne simple.

**Quand** : tu intègres **plusieurs brokers** sans toucher à ton code métier (OMS/stratégies).


In [ ]:

class BrokerA:
    def post(self, symbol, qty): return f"A sent {qty} {symbol}"

class BrokerB:
    def send_order(self, instrument, amount): return f"B OK {amount} {instrument}"

class OMSAdapter:
    def __init__(self, vendor): self.vendor = vendor
    def send(self, symbol, qty):
        if hasattr(self.vendor, "post"):
            return self.vendor.post(symbol, qty)
        if hasattr(self, "send_order"):  # bug volontaire ?
            pass
        if hasattr(self.vendor, "send_order"):
            return self.vendor.send_order(symbol, qty)
        raise ValueError("Unsupported vendor")

print(OMSAdapter(BrokerA()).send("EURUSD", 10))
print(OMSAdapter(BrokerB()).send("EURUSD", 10))



> **Exercice rapide** : repère le **bug** ci‑dessus et corrige‑le (indice : condition sur `send_order`).  
Astuce : ajoute des **tests** qui comparent la sortie attendue pour chaque broker.



## 4) Facade (Façade)

**Idée** : offrir une **API simple** par‑dessus un sous‑système (risk, router, logs, audit).

**Quand** : tu veux manipuler l’OMS sans t’exposer à sa complexité interne.


In [ ]:

class Risk:
    def check(self, side, qty): return qty > 0 and qty <= 1_000

class Router:
    def route(self, symbol, side, qty): return f"ROUTE {side} {symbol} x{qty}"

class OMSFacade:
    def __init__(self, risk: Risk, router: Router):
        self.risk, self.router = risk, router
    def send_order(self, symbol, side, qty):
        if not self.risk.check(side, qty):
            return "REJECT risk"
        return self.router.route(symbol, side, qty)

oms = OMSFacade(Risk(), Router())
print(oms.send_order("EURUSD", "BUY", 100))
print(oms.send_order("EURUSD", "BUY", 5000))  # rejeté



## 5) Singleton (avec parcimonie)

**Idée** : **une seule instance** (ex : config globale).  
**Attention** : complique les tests & la concurrence → privilégier **Dependency Injection**.


In [ ]:

class Config:
    _instance = None
    def __new__(cls, *a, **k):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

c1, c2 = Config(), Config()
print("Singleton?", c1 is c2)



**Alternatives** :
- Objet **module‑level** (simple) ; ou passer la config comme **paramètre**.



## 6) Dependency Injection (DI) — style Pythonic

**Idée** : passer les dépendances (providers, services) **en paramètre** plutôt que de les aller chercher dans des singletons/globals.


In [ ]:

class CurveStub:
    def zero_rate(self, T): return 0.02
class VolStub:
    def sigma(self, T): return 0.15

class PricingEngine:
    def __init__(self, curve_provider, vol_provider):
        self.curve, self.vol = curve_provider, vol_provider
    def price_vanilla(self, T=1.0):
        r = self.curve.zero_rate(T); sigma = self.vol.sigma(T)
        return max(0.0, 100*(1-r*T) - 0.5*sigma)  # jouet

pe = PricingEngine(CurveStub(), VolStub())
print(pe.price_vanilla(1.0))



## 7) Template Method

**Idée** : définir le **squelette** d’un algo dans une classe de base, et laisser les sous‑classes redéfinir des **étapes**.


In [ ]:

from abc import ABC, abstractmethod

class Backtest(ABC):
    def run(self, prices):
        self.before(prices)
        for px in prices:
            self.on_tick(px)
        return self.after()

    def before(self, prices): pass
    @abstractmethod
    def on_tick(self, px): ...
    def after(self): return {}

class BT_MeanReversion(Backtest):
    def __init__(self): self.pos = 0
    def on_tick(self, px):
        # Jouet : change la position selon la parité du prix
        self.pos = 1 if int(px*10) % 2 == 0 else -1
    def after(self): return {"pos": self.pos}

print(BT_MeanReversion().run([100, 101, 100.5]))



## 8) Bonus — Observer asynchrone (idée)

Pour haut débit d’événements : combiner **Observer** + `queue.Queue` **ou** version `asyncio` (producteur async, consommateurs concurrents).



## 9) Mini‑Projet : **OMS Jouet** (exercice guidé)

**Objectif** : écrire un **OrderRouter** qui :
- choisit une stratégie de routing via **Factory** (ex : “best‑price”, “round‑robin”),
- envoie la commande via **Adapter** (unifie BrokerA/BrokerB),
- notifie des auditeurs (logs/risk) via **Observer**,
- expose une **Facade** simple `send_order(symbol, side, qty)`.

**Squelette à compléter** :
```python
# 1) Implémente 2 stratégies de routing (classes StrategyRouting)
# 2) Ecris la factory routing_factory(kind) -> StrategyRouting
# 3) Corrige l'Adapter pour qu'il n'ait plus le bug (section 3)
# 4) Ecris un Subject "Bus" + 2 observers (AuditObserver, RiskObserver)
# 5) Compose tout dans une classe OMSFacadeV2(...).send_order(...)
# 6) Ecris 3 tests simples (assert) : A) vendor A, B) vendor B, C) risk reject
```



---
## 📌 Fiche mémo (révision express)

- **Strategy** : changer l’algo **sans toucher** au reste.
- **Factory** : instancier à la volée selon un **paramètre**.
- **Observer** : diffuser des **ticks**/événements à plusieurs consommateurs.
- **Adapter** : unifier des **APIs externes** (brokers/market data).
- **Facade** : API **simple** sur un sous‑système complexe (risk, router).
- **Singleton** : rare, préférer **DI**.
- **DI** : dépendances passées en **paramètre** → tests faciles.
- **Template Method** : squelette d’algo avec étapes surchargées.
